# Get SDSS photometry for CGM$^2$ galaxy catalog

I am going to try `astroquery`

In [14]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.table import Table
import astropy.units as u
import astropy
import numpy as np

#### Read the CGM$^2$ galaxy `ra` and `dec` from an `ascii` table I made

In [15]:
cgmsqrd_radec_table = Table.read('cgmsquared_gal_radec.txt', format='ascii')
cgmsqrd_radec_table

ID,RA,DEC
int64,float64,float64
146,9.591402,2.08156
159,9.591022,2.06473
293,9.59142,2.10502
343,9.590647,2.04932
393,9.591168,2.10232
432,9.59091,2.09427
456,9.590219,2.04628
480,9.5904,2.07028
497,9.590591,2.09556


#### Convert the units to degrees. RA starts off in hourangle

In [16]:
ra = (cgmsqrd_radec_table['RA']*u.hourangle).to(u.degree)
dec = cgmsqrd_radec_table['DEC']*u.degree

In [17]:
pos = coords.SkyCoord(ra, dec, frame='icrs')
pos

<SkyCoord (ICRS): (ra, dec) in deg
    [(143.87103,  2.08156), (143.86533,  2.06473), (143.8713 ,  2.10502),
     ..., (164.93835, 14.7277 ), (164.96535, 14.72536),
     (164.9001 , 14.72659)]>

#### What fields do we want from SDSS? dered?

http://cas.sdss.org/dr7/en/help/browser/description.asp?n=PhotoObj&t=V

In [18]:
fields = ['ra', 'dec', 'u', 'err_u', 'g', 
          'err_g', 'r', 'err_r', 
          'i', 'err_i', 'z', 'err_z']

# add the dereddened mags just in case
dered_fields = ['dered_u', 'dered_g, ''dered_r', 'dered_i', 'dered_z']

fields += dered_fields

#### Run on one object to see how this works

In [6]:
sql_query = SDSS.query_crossid(pos, photoobj_fields=fields, get_query_payload=True)

In [7]:
# This will take too long and result in a broken pipe
# xid = SDSS.query_crossid(pos, photoobj_fields=['u', 'g', 'r', 'i', 'z'], timeout=100000)

#### Have to run a query on each object (!!!) since cant authorize with casjobs yet. Outputs to a list of tables

In [8]:
out_table = []
for coord in pos:
    xid = SDSS.query_crossid(coord, photoobj_fields=fields)
    out_table.append(xid)

/anaconda3/lib/python3.6/site-packages/astroquery/sdss/core.py:856: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


In order to combine all the tables, need to get rid of the `None` elements in the list which come when there is no object in sdss. Convert to numpy as array, remove the Nones, convert back to list and stack all of em up. 

In [11]:
np_outtable = np.asarray(out_table)
good_elements = np_outtable[~(np_outtable == None)]

# do this once to see if it works
astropy.table.vstack([good_elements[2], good_elements[3]])

/anaconda3/lib/python3.6/site-packages/astropy/table/table.py:2653: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


obj_id,objID,ra,dec,u,err_u,g,err_g,r,err_r,i,err_i,z,err_z,dered_u,dered_g,dered_r,dered_i,dered_z,obj_id1,type
bytes5,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes6
obj_0,1237651753994486409,143.859394665831,2.04948945809487,23.28284,0.800755,22.65892,0.1988366,22.73966,0.3241725,22.74787,0.4746296,21.3777,0.6201465,23.06783,22.50072,22.62492,22.66087,21.31602,1237651753994486409,GALAXY
obj_0,1237651753994486683,143.850890875506,2.06218521755712,22.61179,0.5367773,22.64256,0.2250924,22.74952,0.3831772,21.28734,0.155536,21.06413,0.5619511,22.39552,22.48343,22.6341,21.19983,21.00208,1237651753994486683,GALAXY


In [12]:
sdss_ugriz_table = astropy.table.vstack(good_elements.tolist(), join_type='exact')

Save the file so we dont have to do this again

In [13]:
sdss_ugriz_table.write("sdss_ugriz_table.fits", overwrite=True)